In [83]:
# Importing libraries
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D, Conv3D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# Our dictionary will contain only of the top 7000 words appearing most frequently

# Now we split our data-set into training and test data

# Looking at the nature of training data

In [6]:
import imports as ii
import functions as f
import preprocessing as pp
import neuralnetworks as nn
import trainRNN as trainRNN

In [92]:
PATH = {}
PATH["dataset_classification"] = "dataset/classification/"
PATH["dataset_labeling"] = "dataset/seq_labeling/"
PATH["music_reviews_train"] = PATH["dataset_classification"] + "music_reviews_train.json.gz"
PATH["music_reviews_dev"] = PATH["dataset_classification"] + "music_reviews_dev.json.gz"
PATH["music_reviews_test"] = PATH["dataset_classification"] + "music_reviews_test.json.gz"
train = f.readJson(PATH["music_reviews_train"])
test = f.readJson(PATH["music_reviews_dev"])
test_true = f.readJson(PATH["music_reviews_test"])

Number of data:  100000
Number of data:  10000
Number of data:  10000


In [116]:
X_train, y_train, train_idx, train_missing_idx = f.json_divide(train)
X_dev, y_dev, test_idx, test_missing_idx = f.json_divide(test)
X_test, y_test, test_idx, test_missing_idx = f.json_divide(test)

In [117]:
combination = [1, 1, 1, 1, 1, 1]
combination = [1, 1, 0, 1, 1, 0]
combination = [0, 1, 0, 0, 0, 0]
data_sets, y_train, y_test = f.grid_search_retrain(X_train, X_test, y_train, y_test, combination)

In [118]:
X_train = data_sets[0][1]
X_test = data_sets[0][2]

In [119]:
tokenizer = pp.tokenizer_init(X_train, X_test, X_test)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [120]:
vocabulary_size = len(tokenizer.word_counts)

In [121]:
# Padding the data samples to a maximum review length in words
max_words = 100
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
# Building the CNN Model
model = Sequential()      # initilaizing the Sequential nature for CNN model
# Adding the embedding layer which will take in maximum of 450 words as input and provide a 32 dimensional output of those words which belong in the top_words dictionary
model.add(Embedding(vocabulary_size+1, 32, input_length=max_words))
model.add(Conv1D(32, 1, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [122]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 100, 32)           2336448   
                                                                 
 conv1d_10 (Conv1D)          (None, 100, 32)           1056      
                                                                 
 max_pooling1d_10 (MaxPoolin  (None, 50, 32)           0         
 g1D)                                                            
                                                                 
 flatten_10 (Flatten)        (None, 1600)              0         
                                                                 
 dense_20 (Dense)            (None, 250)               400250    
                                                                 
 dense_21 (Dense)            (None, 1)                 251       
                                                     

In [123]:
sent_dict = {"positive": 1, "negative": 0}
y_train = pp.sentiment_converter(y_train, sent_dict)
y_dev = pp.sentiment_converter(y_dev, sent_dict)
y_test = pp.sentiment_converter(y_test, sent_dict)

In [124]:
import numpy as np
X_test = np.array(X_test)
X_train = np.array(X_train)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [125]:
# Fitting the data onto model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=4, batch_size=100, verbose=1)
# Getting score metrics from our model
scores = model.evaluate(X_test, y_test, verbose=1)
# Displays the accuracy of correct sentiment prediction over test data
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/4
1000/1000 [==============================] - 39s 39ms/step - loss: 0.2846 - accuracy: 0.8815 - val_loss: 0.2363 - val_accuracy: 0.9103
Epoch 2/4
1000/1000 [==============================] - 46s 46ms/step - loss: 0.1796 - accuracy: 0.9340 - val_loss: 0.2529 - val_accuracy: 0.9025
Epoch 3/4
1000/1000 [==============================] - 48s 48ms/step - loss: 0.1284 - accuracy: 0.9552 - val_loss: 0.2895 - val_accuracy: 0.8949
Epoch 4/4
313/313 [==============================] - 1s 3ms/step - loss: 0.3720 - accuracy: 0.8895
Accuracy: 88.95%
